# CSV to XML

In [33]:
!pwd

/Users/premnathknarayanan/programming/Amity/HESA_Compliance


In [34]:
from datetime import datetime

def convert_date_format(date_str):
    return date_str.split(' ')[0]
    try:
        # Parse the date from MM/DD/YYYY format
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        
        # Format the date to YYYY-MM-DD format
        new_date_str = date_obj.strftime('%Y-%m-%d')
        return new_date_str
    except ValueError as e:
        return f"Error: {e}"

def format_with_leading_zero(value):
    value = int(value)
    # Check if the value is a single-digit integer
    if isinstance(value, int) and 0 <= value <= 9:
        return f"{value:02d}"  # Format with leading zero
    else:
        return str(value)  # Convert to string without leading zero

def remove_specific_chars(input_string):
    chars_to_remove = ' -;=?-~ -ɏḀ-ỿ–'
    replacements = {'ă': 'a'}  # Characters to be replaced
    # Create a translation table for replacements
    translation_table = str.maketrans(replacements)

    # Replace specified characters
    replaced_string = input_string.translate(translation_table)
    replaced_characters = ''.join(char for char in replaced_string if char not in chars_to_remove)

    #print(f'{replaced_string}, {replaced_characters}')

    #if replaced_characters != input_string:
        #print(f'{input_string} to {replaced_characters}')
    
    # Check if the input string is empty
    if not replaced_characters:
        replaced_characters = 'NoSurname'    
    return replaced_characters

In [38]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import math

file_name = 'Dec1_Student_data_to_HESA_v6.parquet'
# Read the CSV file
csv_data = pd.read_parquet(file_name) #pd.read_excel(csv_file_path, sheet_name='Dec1_Student_data_to_HESA_v3')

# Create the root XML element
root = ET.Element("DataFutures")

# Load the CSV file
#module_df = pd.read_csv('module_id.csv')
# Number of 'M' columns per group
m_cols = ['MODINSTID', 'MODID', 'MODINSTENDDATE', 'MODINSTSTARTDATE']
num_groups = 8
# Split and concatenate
all_dfs = []
for i in range(num_groups):
    pre_cols = [f'{m}{i+1}' for m in m_cols]
    cols = ['SID'] + pre_cols
    temp_df = csv_data[cols].copy(deep=True)
    temp_df.columns = ['SID'] + m_cols  # Rename columns to standard names
    all_dfs.append(temp_df)

# Concatenate all DataFrames
converted_module_df = pd.concat(all_dfs, ignore_index=True)

# Saving the transformed data to a new CSV file
converted_module_df.to_csv('transformed_file.csv', index=False)

ssn_of_all_students = []
# Applying the detailed association mapping for all students
sid_count = 1
for _, row in csv_data.iterrows():
    sid_int_val = row['SID']
    if len(str(sid_int_val).strip()) <= 0:
        print('Empty SID found')
        continue
    print(f'{sid_count}. SID: {sid_int_val}')
        
    student = ET.SubElement(root, "Student")

    # Direct mappings
    ET.SubElement(student, "SID").text = str(sid_int_val) if pd.notna(sid_int_val) else None
    ET.SubElement(student, "BIRTHDTE").text = convert_date_format(str(row['BIRTHDTE'])) if pd.notna(row['BIRTHDTE']) else None
    ET.SubElement(student, "FNAMES").text = (str(row['FNAMES'])) if pd.notna(row['FNAMES']) else None
    ET.SubElement(student, "GENDERID").text = format_with_leading_zero(int(row['GENDERID'])) if pd.notna(row['GENDERID']) else None
    ET.SubElement(student, "NATION").text = (str(row['NATION'])) if pd.notna(row['NATION']) else None
    ET.SubElement(student, "RELIGION").text = (str(int(row['RELIGION']))) if pd.notna(row['RELIGION']) else None    
    ET.SubElement(student, "SEXID").text = format_with_leading_zero(int(row['SEXID'])) if pd.notna(row['SEXID']) else None
    if pd.notna(row['SSN']):
        ET.SubElement(student, "SSN").text = (str((row['SSN'])))
        ssn_of_all_students.append(row['SSN'])
        
    ET.SubElement(student, "SURNAME").text = remove_specific_chars(str(row['SURNAME'])) if pd.notna(row['SURNAME']) else 'NoSurname'
    ET.SubElement(student, "TTACCOM").text = format_with_leading_zero((row['TTACCOM'])) if pd.notna(row['TTACCOM']) else None            
    ET.SubElement(student, "TTPCODE").text = (str(row['TTPCODE'])) if pd.notna(row['TTPCODE']) else None        
    
    # Nested elements
    disability = ET.SubElement(student, "Disability")
    ET.SubElement(disability, "DISABILITY").text = format_with_leading_zero(row['DISABILITY']) if pd.notna(row['DISABILITY']) else None

    engagement = ET.SubElement(student, "Engagement")
    ET.SubElement(engagement, "NUMHUS").text = str(row['NUMHUS']) if pd.notna(row['NUMHUS']) else None
    if pd.notna(row['ENGEXPECTEDENDDATE']):
        ET.SubElement(engagement, "ENGEXPECTEDENDDATE").text = convert_date_format(str(row['ENGEXPECTEDENDDATE']))
    engStartDate = convert_date_format(str(row['ENGSTARTDATE'])) if pd.notna(row['ENGSTARTDATE']) else None
    ET.SubElement(engagement, "ENGSTARTDATE").text = engStartDate
    ET.SubElement(engagement, "FEEELIG").text = format_with_leading_zero((row['FEEELIG'])) if pd.notna(row['FEEELIG']) else None
    if pd.notna(row['RCSTDNT']):
        ET.SubElement(engagement, "RCSTDNT").text = format_with_leading_zero(int(row['RCSTDNT']))
    #entry_profile = ET.SubElement(engagement, "EntryProfile")
    #entryQualificationAward = ET.SubElement(entry_profile, "EntryQualificationAward")
    #ET.SubElement(entryQualificationAward, "ENTRYQUALAWARDID").text = str(row['QUALAWARDID']) if pd.notna(row['QUALAWARDID']) else None
    #ET.SubElement(entryQualificationAward, "ENTRYQUALAWARDRESULT").text = str(row['QUALAWARDRESULT']) if pd.notna(row['QUALAWARDRESULT']) else None
    #ET.SubElement(entryQualificationAward, "QUALTYPEID").text = str(row['QUALID']) if pd.notna(row['QUALID']) else None

    #entryQualificationSubject = ET.SubElement(entryQualificationAward, "EntryQualificationSubject")    
    #ET.SubElement(entryQualificationSubject, "SUBJECTID").text = str(row['SUBJECTID']) if pd.notna(row['SUBJECTID']) else None

    if pd.notna(row['ENGENDDATE']):
        leaver = ET.SubElement(engagement, "Leaver")
        ET.SubElement(leaver, "ENGENDDATE").text = convert_date_format(str(row['ENGENDDATE'])) if pd.notna(row['ENGENDDATE']) else None
        if pd.notna(row['RSNENGEND']):
            ET.SubElement(leaver, "RSNENGEND").text = format_with_leading_zero(int(row['RSNENGEND']))

    #studentAccreditationAim = ET.SubElement(engagement, "StudentAccreditationAim")
    #ET.SubElement(studentAccreditationAim, "STUACCID").text = str(row['STUACCID']) if pd.notna(row['STUACCID']) else None

    if pd.notna(row['SCSSTARTDATE']):
        studentCourseSession = ET.SubElement(engagement, "StudentCourseSession")
        ET.SubElement(studentCourseSession, "SCSESSIONID").text = str(row['SCSESSIONID']) if pd.notna(row['SCSESSIONID']) else None
        ET.SubElement(studentCourseSession, "COURSEID").text = str(row['COURSEID']) if pd.notna(row['COURSEID']) else None
        ET.SubElement(studentCourseSession, "INVOICEFEEAMOUNT").text = str(int(row['INVOICEFEEAMOUNT'])) if pd.notna(row['INVOICEFEEAMOUNT']) else None
        ET.SubElement(studentCourseSession, "INVOICEHESAID").text = str(int(row['INVOICEHESAID'])) if pd.notna(row['INVOICEHESAID']) else None
        ET.SubElement(studentCourseSession, "RSNSCSEND").text = format_with_leading_zero(int(row['RSNSCSEND'])) if pd.notna(row['RSNSCSEND']) else None        
        if pd.notna(row['SCSENDDATE']):
            ET.SubElement(studentCourseSession, "SCSENDDATE").text = convert_date_format(str(row['SCSENDDATE']))
        ET.SubElement(studentCourseSession, "SCSFEEAMOUNT").text = str(int(row['SCSFEEAMOUNT'])) if pd.notna(row['SCSFEEAMOUNT']) else None
        ET.SubElement(studentCourseSession, "SCSMODE").text = format_with_leading_zero((row['SCSMODE'])) if pd.notna(row['SCSMODE']) else None
        if pd.notna(row['SCSSTARTDATE']):
            ET.SubElement(studentCourseSession, "SCSSTARTDATE").text = convert_date_format(str(row['SCSSTARTDATE'])) if pd.notna(row['SCSSTARTDATE']) else ''
            if pd.notna(row['SESSIONYEARID']):
                ET.SubElement(studentCourseSession, "SESSIONYEARID").text = str(row['SESSIONYEARID'])
        #if pd.notna(row['STULOAD']):
        #    ET.SubElement(studentCourseSession, "STULOAD").text = str(row['STULOAD'])
        if pd.notna(row['YEARPRG']) and pd.notna(row['SCSSTARTDATE']):
            yearprg = row['YEARPRG']
            ET.SubElement(studentCourseSession, "YEARPRG").text = str(int(yearprg))

        fundingAndMonitoring = ET.SubElement(studentCourseSession, "FundingAndMonitoring")
        ET.SubElement(fundingAndMonitoring, "ELQ").text = format_with_leading_zero(int(row['ELQ'])) if pd.notna(row['ELQ']) else '09'
        ET.SubElement(fundingAndMonitoring, "FUNDLENGTH").text = format_with_leading_zero(int(row['FUNDLENGTH'])) if pd.notna(row['FUNDLENGTH']) else '02'        
        ET.SubElement(fundingAndMonitoring, "NONREGFEE").text = '01'
    
        if pd.notna(row['FUNDINGBODY']):
            fundingBody = ET.SubElement(studentCourseSession, "FundingBody")
            ET.SubElement(fundingBody, "FUNDINGBODY").text = str(int(row['FUNDINGBODY']))
        
        student_specific_module_df = converted_module_df[converted_module_df['SID'] == sid_int_val]
        if len(student_specific_module_df) > 0:
            count = 1
            for __, module_row in student_specific_module_df.iterrows():
                if isinstance(module_row['MODINSTID'], float):
                    continue
                if pd.notna(module_row['MODID']):
                    moduleInstance = ET.SubElement(studentCourseSession, "ModuleInstance")
                    ET.SubElement(moduleInstance, "MODINSTID").text = str(sid_int_val).strip()+str(module_row['MODINSTID']).strip()+str(count).strip()
                    ET.SubElement(moduleInstance, "MODID").text = str(module_row['MODID']).strip() if pd.notna(module_row['MODID']) else None
                    ET.SubElement(moduleInstance, "MODINSTENDDATE").text = convert_date_format(str(module_row['MODINSTENDDATE'])) if pd.notna(module_row['MODINSTENDDATE']) else None
                    ET.SubElement(moduleInstance, "MODINSTSTARTDATE").text = convert_date_format(str(module_row['MODINSTSTARTDATE'])) if pd.notna(module_row['MODINSTSTARTDATE']) else None
                count = count + 1
        else:
            print(f'{sid_int_val} without modules')

        referencePeriodStudentLoad = ET.SubElement(studentCourseSession, "ReferencePeriodStudentLoad")
        ET.SubElement(referencePeriodStudentLoad, "REFPERIOD").text = format_with_leading_zero(str(row['REFPERIOD']))
        ET.SubElement(referencePeriodStudentLoad, "YEAR").text = str(row['YEAR'])
        if pd.notna(row['RPSTULOAD']):        
            ET.SubElement(referencePeriodStudentLoad, "RPSTULOAD").text = str(row['RPSTULOAD'])
        if row['COURSEID']=='8N101':
            print(f'{sid_count}.PhD student: {sid_int_val}')

        if pd.notna(row['STATUSVALIDFROM']):
            sessionStatus = ET.SubElement(studentCourseSession, "SessionStatus")        
            ET.SubElement(sessionStatus, "STATUSVALIDFROM").text = convert_date_format(str(row['STATUSVALIDFROM']))
            ET.SubElement(sessionStatus, "STATUSCHANGEDTO").text = format_with_leading_zero((row['STATUSCHANGEDTO']))
        if pd.notna(row['SCSSTARTDATE']):
            studyLocation = ET.SubElement(studentCourseSession, "StudyLocation")
            ET.SubElement(studyLocation, "STUDYLOCID").text = str(row['STUDYLOCID']) if pd.notna(row['STUDYLOCID']) else None
            ET.SubElement(studyLocation, "STUDYPROPORTION").text = str(row['STUDYPROPORTION']) if pd.notna(row['STUDYPROPORTION']) else None
            ET.SubElement(studyLocation, "VENUEID").text = str(row['VENUEID']) if pd.notna(row['VENUEID']) else None            
    
            if row['COURSEID']=='8N101':
                supervisorAllocation = ET.SubElement(studentCourseSession, "SupervisorAllocation")
                ET.SubElement(supervisorAllocation, "SUPALLID").text = str(row['SUPALLID']) if pd.notna(row['SUPALLID']) else None                
                #ET.SubElement(supervisorAllocation, "REF2021UNITOFASSESSMENT").text = str(row['REF2021UNITOFASSESSMENT']) if pd.notna(row['REF2021UNITOFASSESSMENT']) else None                
                ET.SubElement(supervisorAllocation, "SUPALLPROP").text = str(row['SUPALLPROP']) if pd.notna(row['SUPALLPROP']) else None
                
                
    sid_count += 1

# Read the CSV file
csv_data = csv_data = pd.read_parquet(file_name)

venue = ET.SubElement(root, "Venue")
ET.SubElement(venue, "VENUEID").text    = "A"
ET.SubElement(venue, "POSTCODE").text   = "WC1E 7HX"
ET.SubElement(venue, "VENUENAME").text  = "Birkbeck College, University of London"
ET.SubElement(venue, "VENUEUKPRN").text = "10032036"

# Applying the detailed association mapping for all students
for _, row in csv_data.iterrows():
    break
    module = ET.SubElement(root, "Module")

    # Direct mappings
    ET.SubElement(module, "MODID").text = str(row['MODID']) if pd.notna(row['MODID']) else None
    ET.SubElement(module, "FTE").text = (str(row['FTE'])) if pd.notna(row['FTE']) else None
    ET.SubElement(module, "MTITLE").text = (str(row['MTITLE'])) if pd.notna(row['MTITLE']) else None
    
    engagement = ET.SubElement(module, "ModuleCostCentre")
    ET.SubElement(engagement, "COSTCN").text = str(row['COSTCN']) if pd.notna(row['COSTCN']) else None
    ET.SubElement(engagement, "COSTCNPROPORTION").text = (str(row['COSTCNPROPORTION'])) if pd.notna(row['COSTCNPROPORTION']) else None
    #break

# Path to save the output XML file
output_xml_file_path = 'Student_Data_HESA_all_V7.xml'

# Save the XML to a file
tree = ET.ElementTree(root)
tree.write(output_xml_file_path)
print('All generation completed.')

1. SID: 1713535129638
1.PhD student: 1713535129638
2. SID: 1813538075939
2.PhD student: 1813538075939
3. SID: 1813531060015
4. SID: 1813531060048
5. SID: 1813531060093
6. SID: 18100320361060104
7. SID: 1813531060244
8. SID: 1913530260011
9. SID: 1913530260066
10. SID: 1913530260088
11. SID: 1913530260136
12. SID: 19100320360260166
13. SID: 1913530260192
14. SID: 1913530260206
15. SID: 19100320360260247
16. SID: 1913530960018
17. SID: 1913530960030
18. SID: 1913530960041
19. SID: 19100320360960048
20. SID: 1913530960063
21. SID: 1913530960074
22. SID: 1913530960085
23. SID: 2013530260017
24. SID: 2013530260028
25. SID: 2013530260039
26. SID: 2013530260040
27. SID: 2013530260051
28. SID: 2013530260062
29. SID: 2013530260073
30. SID: 2013530260084
31. SID: 2013530260095
32. SID: 2013530260109
33. SID: 2013530960014
34. SID: 2013530960025
35. SID: 2013530960047
36. SID: 2013530960058
37. SID: 2013530960069
38. SID: 2013530960070
39. SID: 2013530960081
40. SID: 2013530960092
41. SID: 201353

In [39]:
ssn_of_all_students

['SFDU17495923M',
 'SFDU18500629G',
 'SFDU18504225L',
 'SFDU18409076G',
 'SFDU18501393E',
 'SFDU18512842P',
 'SFDU18531533W',
 'SFDU18527088P',
 'SFDU15508202D',
 'SFDU18514689J',
 'SFDU18544389C',
 'SFDU18532981L',
 'SFDU18529785E',
 'SFDU19504066W',
 'SFDU19496530K',
 'SFDU19470291X',
 'SFDU19470291X',
 'SFDU19470142R',
 'SFDU18513390U',
 'SFDU19505784H',
 'SFDU17521549V',
 'SFDU19509310K',
 'SFDU19508615S',
 'SFDU20111699M',
 'SFDU19520837C',
 'SFDU17542185P',
 'SFDU20117936T',
 'SFDU19527341W',
 'SFDU20535729R',
 'SFDU20505347A',
 'SFDU20504179A',
 'SFDU20504174L',
 'SFDU20520076V',
 'SFDU20526078H',
 'SFDU17508616G',
 'SFDU20518445C',
 'SFDU20510380Y',
 'SFDU20507128C',
 'SFDU20461207H',
 'SFDU20469919B',
 'SFDU20526639T',
 'SFDU17433122Z',
 'SFDU20514541D',
 'EURS20136512G',
 'SFDU20501911U',
 'SFDU20531311A',
 'SFDU16544392G',
 'SFDU16544392G',
 'SFDU15566602H',
 'SFDU20562333P',
 'SFDU20581531C',
 'SFDU20583539Z',
 'SFDU20563886P',
 'SFDU20579745V',
 'SFDU20563945A',
 'SFDU2056

In [40]:
duplicates = {}
for ssn in ssn_of_all_students:
    if ssn in duplicates:
        duplicates[ssn] += 1
    else:
        duplicates[ssn] = 1

# Filter duplicates
duplicates = {ssn: count for ssn, count in duplicates.items() if count > 1}

print(duplicates)

{'SFDU19470291X': 2, 'SFDU16544392G': 2}


In [36]:
from lxml import etree

def validate_xml(xsd_path, xml_path):
    # Load and parse the XSD file
    with open(xsd_path, 'r') as schema_file:
        schema_doc = etree.XML(schema_file.read())
    schema = etree.XMLSchema(schema_doc)

    # Load and parse the XML file
    with open(xml_path, 'r') as xml_file:
        xml_doc = etree.parse(xml_file)

    # Validate the XML against the schema
    if not schema.validate(xml_doc):
        print("Validation errors were found:")
        for error in schema.error_log:
            print(f"Line {error.line}, Column {error.column}: {error.message}")
    elif schema.validate(xml_doc):
            print("XML is valid against the XSD.")

In [37]:
xml_file = 'Student_Data_HESA_all_V7.xml'  # Replace with your XSD file path
xsd_file = '22056_1_10_0.xsd'  # Replace with your XML file path

validate_xml(xsd_file, xml_file)

XML is valid against the XSD.
